In [1]:
import cv2 as cv
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np

2022-11-17 18:10:04.158054: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
img_size_reduction = 4
vid_num = 0
input_shape = (int(2160/img_size_reduction), int(3840/img_size_reduction), 1)
train_ratio = 0.8
dataset_size = 500
train_frames = int(train_ratio * dataset_size)

In [3]:
x_total = []
y_total = []
test_images = []
back_sub = cv.createBackgroundSubtractorMOG2()
with open(f'imp/videos/labels_0/1 copy.json') as label_file:
    data = json.load(label_file)
for i, k in enumerate(data.keys()):
    if i > dataset_size:
        break
    if "ballCoord" not in data[k]:
        continue
    frame_i = int(k)
    frame = cv.imread(f'imp/videos/frames_0/{vid_num}/{frame_i}.png')
    if i >= train_frames:
        test_images.append(frame)
    frame = back_sub.apply(frame)
    frame = cv.resize(frame, (input_shape[1], input_shape[0]))
    # frame = cv.morphologyEx(frame, cv.MORPH_OPEN, (5, 5))
    # print(frame.shape)
    x, y = data[k]["ballCoord"]
    x_total.append(frame)
    y_total.append([x/img_size_reduction, y/img_size_reduction])

In [4]:
x_total = np.array(x_total)
y_total = np.array(y_total)
x_train, x_test = x_total[:train_frames], x_total[train_frames:]
y_train, y_test = y_total[:train_frames], y_total[train_frames:]

In [5]:
model = models.Sequential([
    Input(shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(2, activation='linear')
])

2022-11-17 18:12:26.363486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
model.fit(x=x_total, y=y_total, epochs=10, batch_size=100, validation_split=0.2)

Epoch 1/10
4/4 [==============================] - 310s 75s/step - loss: 1374.1864 - val_loss: 143.1844
Epoch 2/10
4/4 [==============================] - 283s 71s/step - loss: 389.8387 - val_loss: 610.7706
Epoch 3/10
4/4 [==============================] - 290s 74s/step - loss: 180.8208 - val_loss: 215.7892
Epoch 4/10
4/4 [==============================] - 371s 94s/step - loss: 72.2815 - val_loss: 229.9278
Epoch 5/10
4/4 [==============================] - 385s 97s/step - loss: 43.2101 - val_loss: 163.9501
Epoch 6/10
4/4 [==============================] - 339s 86s/step - loss: 42.8829 - val_loss: 183.4385
Epoch 7/10
4/4 [==============================] - 360s 91s/step - loss: 32.8025 - val_loss: 248.6661
Epoch 8/10
4/4 [==============================] - 326s 82s/step - loss: 26.7700 - val_loss: 308.3380
Epoch 9/10
4/4 [==============================] - 364s 93s/step - loss: 23.0706 - val_loss: 283.1145
Epoch 10/10
4/4 [==============================] - 353s 90s/step - loss: 18.7868 - val_

In [7]:
cv.imshow("frame", x_test[0])
cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

2022-11-17 19:09:19.688 Python[7962:269448] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/s3/v3l8xgqd3xdgc1ll_pmdv1qr0000gn/T/org.python.python.savedState


-1

In [8]:
predictions = model.predict(x_test)

4/4 [==============================] - 13s 3s/step


In [9]:
pred = model.predict(x_test)
for i, image in enumerate(x_test):
    cv.circle(image, tuple(pred[i].astype(int)), 5, (255, 0, 0), 0)
    cv.imshow('frame', image)
    cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

4/4 [==============================] - 11s 2s/step


-1

In [10]:
for i, image in enumerate(x_test):
    print(predictions[i])
    cv.circle(test_images[i], tuple((predictions[i]*img_size_reduction).astype(int)), 20, (255, 0, 0), 5)
    cv.imshow('frame', test_images[i])
    cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

[268.92438 107.25128]
[320.75565 120.90819]
[484.21216 160.50455]
[242.64026  103.787704]
[404.63138 144.35658]
[437.3551  141.62555]
[419.89932 139.85097]
[474.76895 152.61914]
[475.67627 151.85962]
[468.0538  149.89781]
[463.7447  147.85152]
[494.02432 156.9764 ]
[515.14465 164.17276]
[554.6945  179.34233]
[528.39966 173.62675]
[537.842  182.9462]
[543.0088  180.87985]
[568.0393  187.71022]
[551.6118 183.2912]
[520.0305 164.3522]
[555.6048 183.1356]
[527.41864 166.75552]
[560.12024 182.93121]
[555.4019 174.8626]
[537.3472  165.37852]
[561.33215 175.37021]
[517.77374 156.71004]
[521.2238  157.67337]
[478.11536 144.78105]
[524.65393 160.19441]
[553.54565 172.96515]
[529.4513  165.12747]
[557.3949  196.35043]
[536.6214  162.81375]
[558.90515 186.75433]
[530.6116  161.74869]
[545.82666 172.35938]
[554.82275 170.0469 ]
[557.5296  167.41138]
[543.965   165.79971]
[541.5404  166.39261]
[538.57227 162.68896]
[501.12607 146.69498]
[497.99493 147.2081 ]
[488.7584  143.03883]
[511.68372 152.815

3

In [11]:
for i in range(len(x_total)):
    img = x_total[i]
    img = cv.morphologyEx(img, cv.MORPH_OPEN, (4, 4))
    cv.circle(img, tuple(y_total[i].astype(int)), 5, (255, 0, 0), 1)
    cv.imshow("frame", img)
    key = cv.waitKey(0)
    if key == ord('q'):
        cv.destroyAllWindows()
        cv.waitKey(1)
        break



: 